# Usage

## Prepare the data and API key

Get the keys from the environment variables, please use your own keys.

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()
# load up the entries as environment variables
load_dotenv(dotenv_path)
google_key = os.environ.get("google_key")
bing_key = os.environ.get("bing_key")
esri_key = os.environ.get("esri_key")

Load the some data

In [2]:
import pandas as pd
data = pd.read_csv(
    "https://raw.githubusercontent.com/wybert/georouting/main/docs/data/sample_3.csv",
    index_col=0)
data = data[['ZIP_lat', 'ZIP_lon', 'AHA_ID_lat', 'AHA_ID_lon']]
data.head()

,ZIP_lat,ZIP_lon,AHA_ID_lat,AHA_ID_lon
6010,42.376239,-72.605400,42.819978,-73.916518
5333,42.293923,-72.967189,41.753841,-72.682788
7678,42.158520,-72.585325,40.709320,-74.212500


In [3]:
origin = [data.iloc[2]["ZIP_lat"],data.iloc[2]["ZIP_lon"]]
destination = [data.iloc[2]["AHA_ID_lat"],data.iloc[2]["AHA_ID_lon"]]

origins = data[['ZIP_lat', 'ZIP_lon']].values.tolist()
destinations = data[['AHA_ID_lat', 'AHA_ID_lon']].values.tolist()

## Get a route

Get a route between two points is easy, 

In [4]:
from georouting.routers import GoogleRouter
# create a router object with the google_key
router = GoogleRouter(google_key,mode="driving")
# get the route between the origin and destination, this will return a Route object
# this will call the Google Maps API
route = router.get_route(origin, destination)
# Now you can get the distance and duration of the route in meters and seconds
print("Distance: {} meters".format(route.get_distance()))
print("Duration: {} seconds".format(route.get_duration()))

Distance: 260072 meters
Duration: 10279 seconds


You can easily get the distance, duration.

You can also return the routing results in a GeoDataFrame, It will return the distance, duration, speed and the route geometry,

In [5]:
df= route.get_route_geopandas()
df.head()

,distance (m),duration (s),geometry,speed (m/s)
0,30,4,"LINESTRING (-72.58532 42.15852, -72.58520 42.1...",7.500000
1,1146,152,"LINESTRING (-72.58496 42.15851, -72.58504 42.1...",7.539474
2,229,28,"LINESTRING (-72.58001 42.14971, -72.57993 42.1...",8.178571
3,2548,252,"LINESTRING (-72.57958 42.14767, -72.57924 42.1...",10.111111
4,2610,108,"LINESTRING (-72.56706 42.12768, -72.56710 42.1...",24.166667


You can visualize the route in a map,

In [6]:
route.plot_route()

## Get a distance matrix

In [7]:
distance_matrix = router.get_distance_matrix(origins, destinations, append_od=True)
distance_matrix.head()

,orgin_lat,orgin_lon,destination_lat,destination_lon,distance (m),duration (s)
0,42.376239,-72.605400,42.819978,-73.916518,185256,6810
1,42.376239,-72.605400,41.753841,-72.682788,83746,3293
2,42.376239,-72.605400,40.709320,-74.212500,291295,11178
3,42.293923,-72.967189,42.819978,-73.916518,126425,5345
4,42.293923,-72.967189,41.753841,-72.682788,83188,4618


## Get distances according OD pairs

Sometimes you may want to get the durations and distances for some specific origin-destination pairs not for all possible combinations between them. you can use the `get_distances_batch` function. 

In [8]:
distances = router.get_distances_batch(origins, destinations, append_od=True)
distances

,origin_lat,origin_lon,destination_lat,destination_lon,distance (m),duration (s)
0,42.376239,-72.605400,42.819978,-73.916518,185256,6810
0,42.293923,-72.967189,41.753841,-72.682788,83188,4618
0,42.158520,-72.585325,40.709320,-74.212500,260074,10281


It will automatically split the OD pairs into batches and get the distance matrix for each batch to avoid the API limit.

## What's more

`georouting` provides a unified API for routing services, you can use the similar code to get the routing results from different routing services like Google Maps, Bing Maps, OSRM, etc.